In [1]:
#IMPORTING stuff including my online repo with useful tools for processing chess files (PGN)

import sys
import os, sys
if os.path.exists("/kaggle/working/ludus_scacchi"):
    print("Repository already exists, skipping clone.")
    sys.path.append("/kaggle/working/ludus_scacchi")
else:
    !git clone https://github.com/paolomostardi/ludus_scacchi.git /kaggle/working/ludus_scacchi
    !pip install chess
    sys.path.append("/kaggle/working/ludus_scacchi")
from Backend.data_pipeline import from_PGN_generate_bitboards
from Backend.data_pipeline import gm_pipeline as pipe
import pandas as pd
import numpy as np

Cloning into '/kaggle/working/ludus_scacchi'...
remote: Enumerating objects: 1121, done.
remote: Counting objects: 100% (249/249), done.
remote: Compressing objects: 100% (158/158), done.
remote: Total 1121 (delta 151), reused 167 (delta 84), pack-reused 872 (from 4)
Receiving objects: 100% (1121/1121), 160.61 MiB | 23.18 MiB/s, done.
Resolving deltas: 100% (612/612), done.
Updating files: 100% (97/97), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 64.8 MB/s eta 0:00:00:00:010:01
  Preparing metadata (setup.py) ... done
  Created wheel for chess: filename=chess-1.11.2-py3-none-any.whl size=147775 sha256=15b37e97575fef53f14d1e5b508faceaf44f07780b4c9d7474e1f6c8bcec3b58
  Stored in directory: /root/.cache/pip/wheels/83/1f/4e/8f4300f7dd554eb8de70ddfed96e94d3d030ace10c5b53d447
Successfully built chess


In [2]:
!mkdir -p /kaggle/working/Backend/model_evaluation

In [3]:
from Backend.model_evaluation.pipeline import model_evaluation as evals


2026-02-09 11:14:51.721340: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1770635691.939051      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1770635692.008105      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1770635692.567990      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770635692.568039      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770635692.568042      55 computation_placer.cc:177] computation placer alr

In [4]:
x = np.load('/kaggle/input/eval-training-2013-chunk-9/my_processed_chunks/2013-01/chunk_6_x.npy')
y = np.load('/kaggle/input/eval-training-2013-chunk-9/my_processed_chunks/2013-01/chunk_6_y.npy')

In [5]:
print(x.shape)
from Backend.model_evaluation import create_dataset_for_simple_positions as pp

(1000000, 15, 8, 8)


In [6]:
pp.main()

Saving the DF in  the directory :  /kaggle/working/Backend/model_evaluation/simple_positions.csv
Saving the following dataset:                                                   FEN  TargetSquare
0                     k7/6R1/7R/8/8/8/8/K7 w - - 0 1            40
1  rnb1kb1r/ppp1pppp/5n2/3q4/8/2N5/PPPP1PPP/R1BQK...            21
2  r1bqkbnr/pp3ppp/2n1p3/3pP3/3p4/2P2N2/PP3PPP/RN...            21
3  r1bqkb1r/pppp1ppp/2n2n2/4p2Q/2B1P3/8/PPPP1PPP/...            32
4  rnbqkbnr/ppp2ppp/4p3/3P4/3P4/8/PPP2PPP/RNBQKBN...            43
5  rn1qkbnr/pp1bpppp/8/1QPp4/8/8/PPP1PPPP/RNB1KBN...            52
6  r1bqk1nr/pppp1Bpp/2n5/2b1p3/4P3/5N2/PPPP1PPP/R...            59
7  rnb1kbnr/ppp3pp/3p1q2/5p2/3PP3/5N2/PPP2PPP/RNB...            34


In [7]:
stuff_to_insert = evals.evaluate_first_half_model('/kaggle/input/fit-model/resnet_2013_01_04.keras',
                                x[:10],
                                y[:10],
                                '/kaggle/working/Backend/model_evaluation/simple_positions.csv'
                               )

2026-02-09 11:15:15.784324: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
/kaggle/working/ludus_scacchi/Backend/model_evaluation/pipeline/simple_positions_test.py:46: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  position,move = i[1][0],i[1][1]
I0000 00:00:1770635718.096189     153 service.cc:152] XLA service 0x7a78580054d0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1770635718.096279     153 service.cc:160]   StreamExecutor device (0): Host, Default Version
I0000 00:00:1770635719.219125     153 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


k7/6R1/7R/8/8/8/8/K7 w - - 0 1 ------------ Expected move:  40  Acutal move:  7
rnb1kb1r/ppp1pppp/5n2/3q4/8/2N5/PPPP1PPP/R1BQKBNR w KQkq - 2 4 ------------ Expected move:  21  Acutal move:  10
r1bqkbnr/pp3ppp/2n1p3/3pP3/3p4/2P2N2/PP3PPP/RNBQKB1R w KQkq - 0 6 ------------ Expected move:  21  Acutal move:  21
r1bqkb1r/pppp1ppp/2n2n2/4p2Q/2B1P3/8/PPPP1PPP/RNB1K1NR w KQkq - 4 4 ------------ Expected move:  32  Acutal move:  14
rnbqkbnr/ppp2ppp/4p3/3P4/3P4/8/PPP2PPP/RNBQKBNR b KQkq - 0 3 ------------ Expected move:  43  Acutal move:  50
rn1qkbnr/pp1bpppp/8/1QPp4/8/8/PPP1PPPP/RNB1KBNR b KQkq - 0 4 ------------ Expected move:  52  Acutal move:  57
r1bqk1nr/pppp1Bpp/2n5/2b1p3/4P3/5N2/PPPP1PPP/RNBQK2R b KQkq - 0 4 ------------ Expected move:  59  Acutal move:  59
rnb1kbnr/ppp3pp/3p1q2/5p2/3PP3/5N2/PPP2PPP/RNBQKB1R b KQkq - 1 5 ------------ Expected move:  34  Acutal move:  57
Positions guessed correctly:  2 , out of   8
-----
testing if the move changes based on the bitboard color
total percent

In [ ]:
model = 

todo 
Qua inserire il modello nel "db" 
nell notebook fit bisogna eliminare tutte le robe di ludus importate. 





In [10]:
from tensorflow.keras.models import load_model

model = load_model("/kaggle/input/fit-model/resnet_2013_01_04.keras")


In [20]:
n = 4
starting_n = 2

model_name = 'experimental_02'
year_month = '2013-01'

for i in range(starting_n, n):
    print("Stating training with chunk number: ",i)
    model.name = model_name + '_chunk_' + str(i) +'_'+ year_month


Stating training with chunk number:  2
Stating training with chunk number:  3


In [8]:
from Backend.model_evaluation import insert_model_info as ins
ins.insert_model_info(
    
)

Could not import model: No module named 'experimental_architecture1'


TypeError: insert_model_info() missing 4 required positional arguments: 'model_func', 'csv_path', 'main_architecture', and 'name_kaggle'